In [20]:
import asyncio
import datetime
import hashlib
import operator
import os
import time
from typing import Any
import sys
import backoff
import pandas as pd
from more_itertools import chunked
from tqdm import tqdm


In [4]:
import asyncio
import os
import sys
import backoff
import pandas as pd
from more_itertools import chunked
from tqdm import tqdm

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from src.core import keychain

import openai
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key=keychain.get("IACS"))

def save_audio(audio_contents, corpus_name, voice, filename):
    for i, audio in enumerate(audio_contents):
        # Check if the directory exists, create it if it doesn't
        os.makedirs(f"audio/{corpus_name}/{voice}", exist_ok=True)
        with open(f"audio/{corpus_name}/{voice}/{voice}_{filename}.wav", "wb") as f:
            f.write(audio)

@backoff.on_exception(backoff.expo, openai.RateLimitError)
async def generate_audio(text: str, voice: str) -> bytes:
    response = await client.audio.speech.create(
        model="tts-1-hd",
        voice=voice,
        input=text,
        response_format="wav"
    )
    return response.content

async def generate_multiple_audios(texts: list[str], batch_size: int = 10, voice: str = "alloy") -> list[bytes]:
    all_audio_contents = []
    for batch in tqdm(list(chunked(texts, batch_size)), desc="Processing batches"):
        tasks = [generate_audio(text, voice) for text in batch]
        batch_results = await asyncio.gather(*tasks)
        all_audio_contents.extend(batch_results)
    return all_audio_contents

async def generate_and_save_audio(corpus_name: str, texts: list[str], files: list[str], batch_size: int = 10):
    voices = ["alloy"]
    for voice in voices:
        audio_contents = await generate_multiple_audios(texts, batch_size, voice)
        save_audio(audio_contents, "cb", voice, files)

async def main():
    cb_data = pd.read_json("../data/cb/annotations.jsonl", lines=True)
    cb_texts = cb_data["turn"].tolist()
    cb_texts = cb_texts[:10]
    files = cb_data["audio_file"].tolist()
    files = files[:10]
    await generate_and_save_audio("cb", cb_texts, files, batch_size=30)

if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'src'

In [27]:
cb_data = pd.read_json("../data/cb/annotations.jsonl", lines=True)
cb_texts = cb_data["turn"]

In [30]:
from src.core import keychain

import openai
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key=keychain.get("IACS"))

voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]

def save_audio(audio_contents, corpus_name, voice):
    for i, audio in enumerate(audio_contents):
        # Check if the directory exists, create it if it doesn't
        os.makedirs(f"audio/{corpus_name}", exist_ok=True)
        with open(f"audio/{corpus_name}/{voice}_audio_{i}.wav", "wb") as f:
            f.write(audio)

@backoff.on_exception(backoff.expo, openai.RateLimitError)
async def generate_audio(text: str, voice: str) -> bytes:
    response = await client.audio.speech.create(
        model="tts-1-hd",
        voice=voice,
        input=text,
        response_format="mp3"
    )
    return response.content

async def generate_multiple_audios(texts: list[str], batch_size: int = 10, voice: str = "alloy") -> list[bytes]:
    all_audio_contents = []
    for batch in tqdm(list(chunked(texts, batch_size)), desc="Processing batches"):
        tasks = [generate_audio(text, voice) for text in batch]
        batch_results = await asyncio.gather(*tasks)
        all_audio_contents.extend(batch_results)
    return all_audio_contents

batch_size = 10  # Adjust this value based on your needs
for voice in voices:
    audio_contents = await generate_multiple_audios(cb_texts, batch_size, voice)
    save_audio(audio_contents, "cb", voice)



In [29]:
batch_size = 10  # Adjust this value based on your needs
for voice in voices:
    audio_contents = await generate_multiple_audios(cb_texts, batch_size, voice)
    save_audio(audio_contents, "cb", voice)


Processing batches: 100%|██████████| 34/34 [06:10<00:00, 10.89s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'audio/cb/audio_0.mp3'

In [32]:
voice = "shimmer"
save_audio(audio_contents, "cb", voice)
